## [Pandas Reference](https://pandas.pydata.org/pandas-docs/stable/index.html)
1. [Find the unique values in a column and then sort them](https://stackoverflow.com/questions/32072076/find-the-unique-values-in-a-column-and-then-sort-them)
2. [How to select rows from a DataFrame based on column values?](https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values)
3. [Selecting multiple columns in a pandas dataframe](https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe)
4. [How to add pandas data to an existing csv file?](https://stackoverflow.com/questions/17530542/how-to-add-pandas-data-to-an-existing-csv-file)
5. [pandas three-way joining multiple dataframes on columns](https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns)

In [ ]:
import os
import pandas as pd
import numpy as np
# import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
print('pandas version: {}'.format(pd.__version__))
print('numpy version: {}'.format(np.__version__))
# print('matplotlib version: {}'.format(mpl.__version__))
print('matplotlib version: {}'.format(sys.modules[plt.__package__].__version__))

In [ ]:
def exportSheet(data, folder, name):
    if not os.path.exists(folder):
        os.makedirs(folder)
    data.to_csv('{}/{}.CSV'.format(folder, name), index = False)

def exportSheetWithIndex(data, folder, name):
    if not os.path.exists(folder):
        os.makedirs(folder)
    data.to_csv('{}/{}.CSV'.format(folder, name), index = True)

def getAccounts(csv):
    expenses = pd.read_csv(csv)
    uniqueAccounts = expenses['Account'].unique()
    uniqueAccounts.sort()
    return uniqueAccounts

def getCategories(csv):
    expenses = pd.read_csv(csv)
    uniqueCategories = expenses['Category'].unique()
    uniqueCategories.sort()
    return uniqueCategories

def getParentCategories(csv):
    expenses = pd.read_csv(csv)
    uniqueParentCategories = expenses['Parent Category'].unique()
    uniqueParentCategories.sort()
    return uniqueParentCategories

def summarizeExpenses(accountexpenses, categorytranslation, outputdir):
    accountsummary = accountexpenses[['Category', 'Amount']].groupby('Category').agg({'Amount': 'sum'})
    exportSheet(accountsummary, outputdir, 'summary')
    summarytrans = pd.merge(accountsummary, categorytranslation, on = 'Category', how = 'outer')
    exportSheet(summarytrans[summarytrans.Amount.notnull()][['CategoryZh', 'Amount']], outputdir, 'summaryTrans')

def summarizeCategories(accountexpenses, categorytranslation, parentcategorytranslation):
    # accountsummary = accountexpenses[['Parent Category', 'Category', 'Amount']].groupby(['Parent Category', 'Category']).agg({'Amount': 'sum'})
    # print(accountsummary)
    categorymerge = pd.merge(accountexpenses, categorytranslation, on = 'Category', how = 'inner')
    # print(categorymerge['Parent Category', 'Category', 'CategoryZh'])
    # print(categorymerge[categorymerge['Date'].notnull()])
    pcatmerge = pd.merge(categorymerge, parentcategorytranslation, on = 'Parent Category', how = 'inner')
    # print(pcatmerge)
    accountsummary = pcatmerge.copy()[['Parent Category Zh',
                                       'CategoryZh',
                                       'Amount']].groupby(['Parent Category Zh',
                                                           'CategoryZh']).agg({'Amount': 'sum'})
    return accountsummary

def parseExpenses(csv, accounts, categorytranslation, folder, subfolder):
    expenses = pd.read_csv(csv)
    outputdir = '{}/{}'.format(folder, subfolder)
    accountexpenses = expenses[expenses['Account'].isin(accounts)]
    '''
    accountsummary = accountexpenses[['Category', 'Amount']].groupby('Category').agg({'Amount': 'sum'})
    # accountsummary['Category'] = accountsummary.index
    # exportSheet(accountsummary[['Category', 'Amount']], outputdir, 'summary')
    exportSheet(accountsummary, outputdir, 'summary')
    summarytrans = pd.merge(accountsummary, categorytranslation, on = 'Category', how = 'outer')
    summarytrans = summarytrans[summarytrans.Amount.notnull()]
    exportSheet(summarytrans[['CategoryZh', 'Amount']], outputdir, 'summaryTrans')
    '''
    summarizeExpenses(accountexpenses, categorytranslation, outputdir)
    
    for category in accountexpenses['Category'].unique():
        datasetraw = accountexpenses[accountexpenses['Category'] == category].sort_values('Date')
        dataset = datasetraw[['Date', 'Description', 'Original Description',
                              'Amount', 'Type', 'Account', 'Memo', 'Pending']]
        dataset.loc['Total'] = pd.Series(dataset['Amount'].sum(), index = ['Amount'])
        exportSheet(dataset, outputdir, category)

def pickExpensesByAccounts(expenses, accounts, categorytranslation, folder, subfolder, accountCategory):
    outputdir = '{}/{}'.format(folder, subfolder)
    accountexpenses = expenses[expenses['Account'].isin(accounts)]
    exportSheet(accountexpenses[['Date', 'Description', 'Original Description',
                                 'Amount', 'Type', 'Parent Category', 'Category',
                                 'Account', 'Memo', 'Pending']], outputdir, accountCategory)

def translateExpenses(expenses, parentcategorytranslation, categorytranslation, folder, subfolder, accountCategory):
    outputdir = '{}/{}'.format(folder, subfolder)
    categorymerge = pd.merge(expenses, categorytranslation, on = 'Category', how = 'inner')
    exportSheet(categorymerge.sort_values(by = ('Date')), outputdir, '{}_categoryZH'.format(accountCategory))
    pcatmerge = pd.merge(categorymerge, parentcategorytranslation, on = 'Parent Category', how = 'inner')
    exportSheet(pcatmerge.sort_values(by = ('Date')), outputdir, '{}_parent_categoryZH'.format(accountCategory))
    pdata = pcatmerge[['Date', 'Description', 'Original Description',
                                 'Amount', 'Type', 'Parent Category Zh', 'CategoryZh',
                                 'Account', 'Memo', 'Pending']].copy()
    pdata['Date'] = pd.to_datetime(pdata.Date)
    exportSheet(pdata.sort_values(by = ('Date')), outputdir, accountCategory)

def monthlyexpenses(expenses, parentcategorytranslation, categorytranslation, folder, subfolder):
    summary = summarizeCategories(expenses, categorytranslation, parentcategorytranslation)
    exportSheetWithIndex(summary, '{}/{}'.format(folder, subfolder), 'summary')
    translateExpenses(expenses, pCatZhDf, catDf, folder, subfolder, 'expenses')

In [ ]:
catDf = pd.read_csv('../work/categories_chinese_translation.csv')

In [ ]:
pCatZhDf = pd.read_csv('../work/parent_categories_chinese_translation.csv')

In [ ]:
Cats08 = pd.DataFrame(getCategories('../work/201908.csv'), columns = ['Category'])
Cats08Trans = pd.merge(Cats08, catDf, on = 'Category', how = 'outer')
# print(Cats08Trans[Cats08Trans['CategoryZh'].isnull()]['Category'].tolist())
# ['Lawn & Garden', 'Shoes']

In [ ]:
with open('../work/categories_chinese_translation.csv', 'a') as f:
    Cats08TransAdd = list(zip(Cats08Trans[Cats08Trans['CategoryZh'].isnull()]['Category'].tolist(),
                              ['草坪與花園','鞋子']))
    Cats08TransAddDf = pd.DataFrame(Cats08TransAdd, columns = ['Category', 'CategoryZh'])
    Cats08TransAddDf.to_csv(f, header = False, index = False)

In [ ]:
exp201908 = pd.read_csv('../work/201908.csv')
monthlyexpenses(exp201908, pCatZhDf, catDf, '../work', '201908')

In [ ]:
pCats = getParentCategories('../work/201908.csv')
print(pCats)
parentCats = pd.DataFrame(getParentCategories('../work/201908.csv'))
print(parentCats.dtypes)
print(parentCats.shape)
print(parentCats[~parentCats.isin(catDf['Category'])].shape)
print(parentCats[parentCats.isin(catDf['Category'])].shape)
parentCategoryTrans = list(zip(pCats.tolist(),['汽車與交通','賬單與公用事業','教育','娛樂','費用',
                                             '金融','餐飲','禮物和捐贈','家庭','收入','投資',
                                             '個人護理','購物','轉帳']))
pCatTranDf = pd.DataFrame(parentCategoryTrans, columns = ['Parent Category', 'Parent Category Zh'])
pCatTranDf.to_csv('../work/parent_categories_chinese_translation.csv', index = False)

In [ ]:
pCats09 = pd.DataFrame(getParentCategories('../work/201909.csv'), columns = ['Parent Category'])
pCats09Trans = pd.merge(pCats09, pCatZhDf, on = 'Parent Category', how = 'outer')
# print(pCats09Trans[pCats09Trans['Parent Category Zh'].isnull()]['Parent Category'].tolist())
# ['Business Services', 'Health & Fitness', 'Kids']

In [ ]:
with open('../work/parent_categories_chinese_translation.csv', 'a') as f:
    pCats09TransAdd = list(zip(pCats09Trans[pCats09Trans['Parent Category Zh']
                                            .isnull()]['Parent Category'].tolist(),
                               ['商業服務','健康與健身','孩子']))
    pCats09TransAddDf = pd.DataFrame(pCats09TransAdd, columns = ['Parent Category', 'Parent Category Zh'])
    pCats09TransAddDf.to_csv(f, header = False, index = False)

In [ ]:
exp201909 = pd.read_csv('../work/201909.csv')
monthlyexpenses(exp201909, pCatZhDf, catDf, '../work', '201909')

In [ ]:
exp201910 = pd.read_csv('../work/201910.csv')
monthlyexpenses(exp201910, pCatZhDf, catDf, '../work', '201910')

In [ ]:
exp201911 = pd.read_csv('../work/201911.csv')
monthlyexpenses(exp201911, pCatZhDf, catDf, '../work', '201911')

In [ ]:
getAccounts('Expenses.csv')
getAccounts('../work/transactions_20191122_past90days.csv')
parseExpenses('Expenses.csv', ['0289 * Business Member Share Savings',
                               '0388 * Business Basic Checking',
                               '5734 * Member Share Savings',
                               '7238 * ',
                               'Adv Plus Banking - 8129',
                               'Annie - 9823',
                               'Bryce - 9470',
                               'Nicole - 9471'], '20191114', 'bankaccounts')
parseExpenses('Expenses.csv', ['Hilton Honors Card',
                               'chunyenwang-8046'], '20191114', 'creditcards')

In [ ]:
print(getAccounts('../work/transactions_20191122_past90days.csv'))
categoryEN = getCategories('../work/transactions_20191122_past90days.csv')
categoryTrans = list(zip(categoryEN.tolist(), ['ATM費用', '酒類和酒吧', '汽車與運輸', '汽車保險', '書籍和用品', '商業服務',
                                 '慈善', '支票', '咖啡店', '信用卡付款', '分紅和上限', '醫生', '教育',
                                 '電子和軟件', '娛樂', '快餐', '費用', '財務費用', '財務顧問', '餐飲',
                                 '汽油', '雜貨店', '健康與健身', '房屋', '房屋裝修', '收入', '利息收入',
                                 '投資', '兒童活動', '手機', '電影和DVD', '音樂', '音樂課', '辦公用品',
                                 '停車', '個人護理', '出租車', '服務和零件', '運輸', '購物', '稅收', '轉帳', '水電費']))
catDf = pd.DataFrame(categoryTrans, columns = ['Category', 'CategoryZh'])
catDf.to_csv('../work/categories_chinese_translation.csv', index = False)

In [ ]:
parseExpenses('../work/transactions_20191122_past90days.csv', ['0289 * Business Member Share Savings',
                               '0388 * Business Basic Checking',
                               '5734 * Member Share Savings',
                               '7238 * ',
                               'Adv Plus Banking - 8129',
                               'Annie - 9823',
                               'Bryce - 9470',
                               'Nicole - 9471'], catDf, '../work/transactions_20191122_past90days', 'bankaccounts')
parseExpenses('../work/transactions_20191122_past90days.csv', ['Hilton Honors Card',
                               'chunyenwang-8046'], catDf, '../work/transactions_20191122_past90days', 'creditcards')

In [ ]:
catDf = pd.read_csv('../work/categories_chinese_translation.csv')
pickExpensesByAccounts('../work/transactions_20191122_past90days.csv',
                       ['0289 * Business Member Share Savings',
                        '0388 * Business Basic Checking',
                        '5734 * Member Share Savings',
                        '7238 * ',
                        'Adv Plus Banking - 8129',
                        'Annie - 9823',
                        'Bryce - 9470',
                        'Nicole - 9471'],
                       catDf, '../work/transactions_20191122_past90days',
                       'bankaccounts', 'bankaccounts')
pickExpensesByAccounts('../work/transactions_20191122_past90days.csv',
                       ['Hilton Honors Card',
                        'chunyenwang-8046'],
                       catDf, '../work/transactions_20191122_past90days',
                       'creditcards', 'creditcards')